In [ ]:
from pyspark.sql.functions import explode, sequence, to_date

fechaInicio = '2000-01-01'
fechaFin = '2025-12-31'

spark.sql(f"select explode(sequence(to_date('{fechaInicio}'), to_date('{fechaFin}'), interval 1 day)) as fecha") \
    .createOrReplaceTempView('vwfecha')


In [ ]:
df = spark.sql("""
select
  year(fecha) * 10000 + month(fecha) * 100 + day(fecha) AS idFecha,
  fecha,
  year(fecha) AS Anio,
  quarter(fecha) AS Trimestre,
  date_format(fecha, 'QQQ') AS TrimestreDesc,
  month(fecha) AS Mes,
  date_format(fecha, 'MMMM') AS MesDesc,
  CASE 
    WHEN month(fecha) = 1 THEN 'Enero'
    WHEN month(fecha) = 2 THEN 'Febrero'
    WHEN month(fecha) = 3 THEN 'Marzo'
    WHEN month(fecha) = 4 THEN 'Abril'
    WHEN month(fecha) = 5 THEN 'Mayo'
    WHEN month(fecha) = 6 THEN 'Junio'
    WHEN month(fecha) = 7 THEN 'Julio'
    WHEN month(fecha) = 8 THEN 'Agosto'
    WHEN month(fecha) = 9 THEN 'Septiembre'
    WHEN month(fecha) = 10 THEN 'Octubre'
    WHEN month(fecha) = 11 THEN 'Noviembre'
    WHEN month(fecha) = 12 THEN 'Diciembre'
  END AS MesDesc_ESP,
  day(fecha) AS dia,
  date_format(fecha, 'EEEE') AS DiaDesc,
  CASE
    WHEN dayofweek(fecha) = 1 THEN 'Domingo'
    WHEN dayofweek(fecha) = 2 THEN 'Lunes'
    WHEN dayofweek(fecha) = 3 THEN 'Martes'
    WHEN dayofweek(fecha) = 4 THEN 'Miércoles'
    WHEN dayofweek(fecha) = 5 THEN 'Jueves'
    WHEN dayofweek(fecha) = 6 THEN 'Viernes'
    WHEN dayofweek(fecha) = 7 THEN 'Sábado'
  END AS DiaDesc_ESP,   
  dayofyear(fecha) AS DiaDelAnio,
  weekofyear(fecha) AS Semana,
  date_format(fecha, 'yyyy-MM') AS Periodo
from
  vwfecha

""")

df.write.mode('overwrite').format("delta").saveAsTable("dimFecha")